## Predictive Maintenance using Machine Learning

Brendon Hall, [Enthought](www.enthought.com)


This notebook is an adaptation of the predictive maintenance notebook found [here](https://github.com/Azure/lstms_for_predictive_maintenance)[1] and [here](https://gallery.azure.ai/Notebook/Predictive-Maintenance-Modelling-Guide-Python-Notebook-1#x_Feature-Engineering)[2].

This notebook explores a machine learning workflow to build a predictive maintenance model.  Predictive maintenance is an important problem in asset heavy industries and so is particularly relevant to the oil and gas industry.  One example would be predicting the failure of electronic submersible pumps used for artificial lift.  Maintenance of these pumps requires workover rigs, and and advance indication of failure would allow time for scheduling the logistics of rig time, etc.

In order to explore the steps necessary to create a predictive maintenance model, we will use a synthetic synthetic turbofan engine degredation dataset generated by NASA Ames Research Center [[3]](https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/).  This is licensed as a [U.S. Government work](https://www.usa.gov/government-works). 

This dataset can be considered to consist of 100 similar machines, which we will use as proxy ESPs.  Each machine starts with different degrees of initial wear and manufacturing variation which is unknown to the user.  This wear and variation is considered normal, i.e., it is not considered a fault condition. There are three operational settings that have a substantial effect on engine performance. These settings are also included in the data. There are 21 sensor measurements in the dataset which is contaminated with sensor noise.

![ESP schematic](images/ESP_diagram.png "Logo Title Text 1")

The engine is operating normally at the start of each time series, and develops a fault at some point during the series. In the training set, the fault grows in magnitude until system failure. In the test set, the time series ends some time prior to system failure. The predictive maintenace problem involves predicting when failure will occur in the test set. Also provided a vector of true Remaining Useful Life (RUL) values for the test data.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

# Setting seed for reproducability
np.random.seed(1234)  
PYTHONHASHSEED = 0
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, recall_score, precision_score

from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

%matplotlib inline

The dataset can be downloaded [here](https://ti.arc.nasa.gov/c/6/).  For this example, we will just use the training data in `train_FD001.txt`, the test data in `test_FD001.txt` and the remaining useful life (RUL) for the test data in `RUL_FD001.txt`.

The data are provided as a text files with 26 columns, separated by spaces.  Each row is a snapshot of data taken during a single operational cycle.  The following chart defines the variables in each column:

|Index | Column Name | Type | Description|
|------|-------------|------|------------|
| 1  |        id   |  Int  | Machine ID         |
| 2  |    cycle  |Int | time, in cycles | 
| 3 | setting1 | Float | Control Setting 1 |
| 4 | setting2 | Float | Control Setting 1 |
| 5 | setting3 | Float | Control Setting 1 |
| 6 | s1 | Float | Sensor Measurement 1 |
| 7 | s2 | Float | Sensor Measurement 2 |
|...|...|...|...|
|26|s21|Float|Sensor Measurement 21|

In [ ]:
# read training data
sensor_column_names = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df = pd.read_csv('data/train_FD001.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = sensor_column_names

In [ ]:
# read test data
test_df = pd.read_csv('data/test_FD001.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = sensor_column_names

In [ ]:
# read ground truth data
truth_df = pd.read_csv('data/RUL_FD001.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)

In [ ]:
train_df = train_df.sort_values(['id','cycle'])
train_df.head()

## Data Preprocessing

When designing any machine learning model, it is critical to understand the overall objective.  In predictive maintenance, we want to predict when an operational machine will fail, so that appropriate steps can be taken in advance.

Predictive maintenance can be done using a variety of machine learning modeling types:

- **Regression:** Predict the Remaining Useful Life (RUL), or Time to Failure (TTF).
- **Binary classification:** Predict if an asset will fail within certain time frame (e.g. days).
- **Multi-class classification:** Predict if an asset will fail in different time windows: E.g., fails in window [1, w0] days; fails in the window [w0+1,w1] days; not fail within w1 days

The assumption is that the machines (ie: pumps) degrade over time, and that degradation is reflected in the machine's sensor measurements.  Ideally a machine learning algorithm can be created to learn how sensor measurements are related to machine failure.

For this example we will consider a binary classification problem, and predict if a machine is going to fail within 30 cycles.   The training data consists of sensor measurements for each machine until it fails.  We will use this to create a column with the RUL of each machine.

In [ ]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

Now create a column called `label1` that indicates if a machine will fail within `w1` cycles (set to 30 for now).

**Bonus:** Create a label for a multiclass classification problem, that predicts if a machine will fail within w1 cycles, fail within w0 cycles, or not fail within a specified window.

In [ ]:
# generate label columns for training data
w1 = 30
train_df['label1'] = np.where(train_df['RUL'] <= w1, 1, 0 )
train_df.head()

The `cycle` column can be used for training.  Normalize that column to provide consistancy across the training set.

In [ ]:
# MinMax normalization
# train_df['cycle_norm'] = train_df['cycle']
# cols_normalize = train_df.columns.difference(['id','cycle','RUL','label1'])
# min_max_scaler = preprocessing.MinMaxScaler()
# norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
#                              columns=cols_normalize, 
#                              index=train_df.index)
# join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
# train_df = join_df.reindex(columns = train_df.columns)
# train_df.head()

Now we prepare the test data set.   We first normalize the test data using the parameters from the MinMax normalization applied on the training data.

Next, we use the ground truth dataset to generate labels for the test data.

In [ ]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [ ]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
test_df.head()

Generate label columns for the test data using the ground truth values.

In [ ]:
# generate label columns w0 and w1 for test data
test_df['label1'] = np.where(test_df['RUL'] <= w1, 1, 0 )
test_df.head()

In [ ]:
# test_df['cycle_norm'] = test_df['cycle']
# norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
#                             columns=cols_normalize, 
#                             index=test_df.index)
# test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
# test_df = test_join_df.reindex(columns = test_df.columns)
# #test_df = test_df.reset_index(drop=True)
# test_df.head()

## Sensor Data

Let's take a look at the sensor data for one machine.  We'll extract the last 50 cycles of data for machine ID=3, just prior to failure. 

In [ ]:
# preparing data for visualizations 
# window of 50 cycles prior to a failure point for engine id 3
engine_id3 = test_df[test_df['id'] == 3]
engine_id3_50cycleWindow = engine_id3[engine_id3['RUL'] <= engine_id3['RUL'].min() + 50]
cols1 = ['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10']
engine_id3_50cycleWindow1 = engine_id3_50cycleWindow[cols1]
cols2 = ['s11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21']
engine_id3_50cycleWindow2 = engine_id3_50cycleWindow[cols2]

In [ ]:
# plotting sensor data for engine ID 3 prior to a failure point - sensors 1-10 
ax1 = engine_id3_50cycleWindow1.plot(subplots=True, sharex=True, figsize=(20,20))

In [ ]:
# plotting sensor data for engine ID 3 prior to a failure point - sensors 11-21 
ax2 = engine_id3_50cycleWindow2.plot(subplots=True, sharex=True, figsize=(20,20))

## Modelling

Now we can proceed to use the training and test data to create a simple classification model, examine the results and determine the best way to improve it.

In [ ]:
X_train = train_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_train = train_df['label1'].values

X_test = test_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_test = test_df['label1'].values

model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

def display_metrics(y_true, y_pred):
    labels = ['No Failure', 'Failure30']
    cm = confusion_matrix(y_true, y_pred)
    print('Confusion matrix\n- x-axis is true labels\n- y-axis is predicted labels')
    print(cm)
    print(classification_report(y_true, y_pred, target_names=labels))

display_metrics(y_test, y_pred)

Now we show the feature importances of the first trained model.

In [ ]:
plt.figure(figsize=(10, 6))
feature_names = test_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).columns.values

labels, importances = zip(*sorted(zip(feature_names, model.feature_importances_), reverse=True, key=lambda x: x[1]))
plt.xticks(range(len(labels)), labels)
_, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.bar(range(len(importances)), importances)
plt.ylabel('Importance')

In predictive maintenance, machine failures are usually rare occurrences in the lifetime of the assets compared to normal operation. This causes an imbalance in the label distribution which usually causes poor performance as algorithms tend to classify majority class examples better at the expense of minority class examples as the total misclassification error is much improved when majority class is labeled correctly. This causes low recall rates although accuracy can be high and becomes a larger problem when the cost of false alarms to the business is very high. To help with this problem, sampling techniques such as oversampling of the minority examples are usually used along with more sophisticated techniques which are not covered in this notebook.

In [ ]:
plt.figure(figsize=(8, 4))
train_df['label1'].value_counts().plot(kind='bar')
plt.xlabel('Failure Status')
plt.ylabel('Count')

Now we can investigate and create features that will be useful for predicting machine failure in an effort to improve the model.

## Feature Engineering

An important step in predictive maintenance applications is feature engineering which requires bringing the different data sources together to create features that best describe a machines's health condition at a given point in time. In the next section, several feature engineering methods are used to create features based on the properties of each data source.

In [ ]:
# pick the feature columns 
sensor_cols = ['s' + str(i) for i in range(1,22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']

sensor_df = train_df[sensor_cols + ['cycle', 'id']]
sensor_df.head()

## Rolling averages

Time series data always comes with time-stamps which makes it suitable for calculating rolling features. A common method is to pick a window size for the lag features to be created and compute rolling aggregate measures such as mean, standard deviation, minimum, maximum, etc. to represent the short term history of the sensor data over the window. In the following, rolling mean and standard deviation of the sensor data over the last 3 cycle window is calculated for every 3 cycles.

You can read more about the rolling function [here](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html?highlight=rolling#pandas.DataFrame.rolling).

In [ ]:
train_3hm_df = sensor_df.groupby('id').rolling(window=3, on='cycle', min_periods=1).mean().reset_index(drop=True)
test_3hm_df = test_df[sensor_cols + ['cycle', 'id']].groupby('id').rolling(window=5, on='cycle', min_periods=1).mean().reset_index(drop=True)


train_3hm_df.drop(['id', 'cycle'], axis=1, inplace=True)
train_3hm_df.columns = [i + 'm_3h' for i in sensor_cols]

test_3hm_df.drop(['id', 'cycle'], axis=1, inplace=True)
test_3hm_df.columns = [i + 'm_3h' for i in sensor_cols]

train_aug_df = pd.concat([train_df, train_3hm_df], axis=1)
test_aug_df = pd.concat([test_df, test_3hm_df], axis=1)

The following cell will compute the 3 cycle rolling standard deviation...

In [ ]:
train_3hsd_df = sensor_df.groupby('id').rolling(window=3, on='cycle', min_periods=1).std().reset_index(drop=True)
test_3hsd_df = test_df[sensor_cols + ['cycle', 'id']].groupby('id').rolling(window=3, on='cycle', min_periods=1).std().reset_index(drop=True)


train_3hsd_df.drop(['id', 'cycle'], axis=1, inplace=True)
train_3hsd_df.columns = [i + 'sd_3h' for i in sensor_cols]

test_3hsd_df.drop(['id', 'cycle'], axis=1, inplace=True)
test_3hsd_df.columns = [i + 'sd_3h' for i in sensor_cols]

train_aug_df = pd.concat([train_aug_df, train_3hsd_df], axis=1).dropna()
test_aug_df = pd.concat([test_aug_df, test_3hsd_df], axis=1).dropna()

In [ ]:
X_train = train_aug_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_train = train_aug_df['label1'].values

X_test = test_aug_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_test = test_aug_df['label1'].values

model2 = GradientBoostingClassifier(random_state=42)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

display_metrics(y_test, y_pred)

## Lagged features

In [ ]:
shift1 = sensor_df.groupby('id').shift(1).reset_index(drop=True)
shift1.drop(['cycle'], axis=1, inplace=True)
shift1.columns = [i + '_lag1' for i in sensor_cols]

shift2 = sensor_df.groupby('id').shift(2).reset_index(drop=True)
shift2.drop(['cycle'], axis=1, inplace=True)
shift2.columns = [i + '_lag2' for i in sensor_cols]

shift3 = sensor_df.groupby('id').shift(3).reset_index(drop=True)
shift3.drop(['cycle'], axis=1, inplace=True)
shift3.columns = [i + '_lag3' for i in sensor_cols]

shift1_test = test_df[sensor_cols + ['cycle', 'id']].groupby('id').shift(1).reset_index(drop=True)
shift1_test.drop(['cycle'], axis=1, inplace=True)
shift1_test.columns = [i + '_lag1' for i in sensor_cols]

shift2_test = test_df[sensor_cols + ['cycle', 'id']].groupby('id').shift(2).reset_index(drop=True)
shift2_test.drop(['cycle'], axis=1, inplace=True)
shift2_test.columns = [i + '_lag2' for i in sensor_cols]

shift3_test = test_df[sensor_cols + ['cycle', 'id']].groupby('id').shift(3).reset_index(drop=True)
shift3_test.drop(['cycle'], axis=1, inplace=True)
shift3_test.columns = [i + '_lag3' for i in sensor_cols]

train_shift_df = pd.concat([train_aug_df,shift1, shift2, shift3], axis=1).dropna()
test_shift_df = pd.concat([test_aug_df,shift1_test, shift2_test, shift3_test], axis=1).dropna()

In [ ]:
X_train = train_shift_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_train = train_shift_df['label1'].values

X_test = test_shift_df.drop(['id', 'cycle', 'RUL', 'label1'], axis=1).values
y_test = test_shift_df['label1'].values

model2 = GradientBoostingClassifier(random_state=42)
model2.fit(X_train, y_train)
y_pred = model2.predict(X_test)

display_metrics(y_test, y_pred)

## Dimensionality Reduction

We have quite a few features after feature engineering, we can try reducing the total number using PCA to improve performance.

In [ ]:
pca = PCA(n_components=20)
pca.fit(X_train)
X_train_p = pca.transform(X_train)
X_test_p = pca.transform(X_test)

model2 = GradientBoostingClassifier(random_state=42)
model2.fit(X_train_p, y_train)
y_pred = model2.predict(X_test_p)

display_metrics(y_test, y_pred)

In predictive maintenance, we are often most concerned with how many of the actual failures were predicted by the model, i.e. the model's recall. (Recall becomes more important as the consequences of false negatives -- true failures that the model did not predict -- exceed the consequences of false positives, viz. false prediction of impending failure.) Below, we compare the recall rates for each failure type for the three models. The recall rates for all components as well as no failure are all above 90% meaning the model was able to capture above 90% of the failures correctly.

## Summary

In this notebook, the steps of implementing a predictive maintenance model is provided using an example scenario where the goal is to predict failures due to certain components of a machine. Typical steps of predictive maintenance such as feature engineering, labelling, training and evaluation are explained using the example data sets.



## References

[1] LSTMS for Predictive Maintenance (https://github.com/Azure/lstms_for_predictive_maintenance)

[2] Predictive Maintenance Guide Python Notebook (https://gallery.azure.ai/Notebook/Predictive-Maintenance-Modelling-Guide-Python-Notebook-1#x_Feature-Engineering)

[1] A. Saxena and K. Goebel (2008). "Turbofan Engine Degradation Simulation Data Set", NASA Ames Prognostics Data Repository (https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/), NASA Ames Research Center, Moffett Field, CA